In [ ]:
## Part I: Preprocessing and EDA
- The data comes from a global e-retailer company, including orders from 2012 to 2015. 
Import the **Orders** dataset and do some basic EDA. 
- For problem 1 to 3, we mainly focus on data cleaning and data visualizations. 
You can use all the packages that you are familiar with to conduct some plots and 
also provide **brief interpretations** about your findings.

### Problem 1: Dataset Import & Cleaning
Check **"Profit"** and **"Sales"** in the dataset, convert these two columns to numeric type. 
### Problem 2: Inventory Management
- Retailers that depend on seasonal shoppers have a particularly challenging job when it comes 
to inventory management. Your manager is making plans for next year's inventory.
- He wants you to answer the following questions:
    1. Is there any seasonal trend of inventory in the company?
    2. Is the seasonal trend the same for different categories?
- ***Hint:*** For each order, it has an attribute called `Quantity` that indicates the number 
    of product in the order. If an order contains more than one product, there will be multiple 
    observations of the same order.
### Problem 3: Why did customers make returns?
- Your manager required you to give a brief report (**Plots + Interpretations**) on returned orders.

	1. How much profit did we lose due to returns each year?
	2. How many customer returned more than once? more than 5 times?
	3. Which regions are more likely to return orders?
	4. Which categories (sub-categories) of products are more likely to be returned?
- ***Hint:*** Merge the **Returns** dataframe with the **Orders** dataframe using `Order.ID`.

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

orders = pd.read_csv('data/Orders.csv')
returns = pd.read_csv('data/Returns.csv')
pd.options.display.max_columns = None

In [ ]:
orders.columns

In [24]:
orders.head()

,Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority
0,40098,CA-2014-AB10015140-41954,11/11/14,11/13/14,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,$221.98,2,0.0,$62.15,40.77,High
1,26341,IN-2014-JR162107-41675,2/5/14,2/7/14,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black","$3,709.40",9,0.1,-$288.77,923.63,Critical
2,25330,IN-2014-CR127307-41929,10/17/14,10/18/14,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID","$5,175.17",9,0.1,$919.97,915.49,Medium
3,13524,ES-2014-KM1637548-41667,1/28/14,1/30/14,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,Berlin,Germany,Western Europe,Europe,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless","$2,892.51",5,0.1,-$96.54,910.16,Medium
4,47221,SG-2014-RH9495111-41948,11/5/14,11/6/14,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,Dakar,Senegal,Western Africa,Africa,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed","$2,832.96",8,0.0,$311.52,903.04,Critical


In [9]:
orders.sample(10)


,Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority
26639,30847,ID-2013-KH1669092-41601,11/23/13,11/28/13,Standard Class,KH-1669092,Kristen Hastings,Corporate,NaN,Manukau City,Auckland,New Zealand,Oceania,Asia Pacific,OFF-SU-2995,Office Supplies,Supplies,"Acme Shears, Serrated",$111.89,4,0.400,-$54.19,7.210,Medium
50102,50375,NI-2012-SP1086095-41090,6/30/12,7/3/12,First Class,SP-1086095,Sung Pak,Corporate,NaN,Ibadan,Oyo,Nigeria,Western Africa,Africa,OFF-BI-2917,Office Supplies,Binders,"Acco Index Tab, Clear",$5.15,2,0.700,-$9.31,1.190,Critical
37054,21973,IN-2015-ES1402027-42064,3/1/15,3/5/15,Standard Class,ES-1402027,Erica Smith,Consumer,NaN,Nantong,Jiangsu,China,Eastern Asia,Asia Pacific,OFF-EN-4432,Office Supplies,Envelopes,"GlobeWeis Business Envelopes, Security-Tint",$39.48,2,0.000,$4.32,2.990,High
4980,9024,MX-2013-BN1151531-41552,10/5/13,10/9/13,Standard Class,BN-1151531,Bradley Nguyen,Consumer,NaN,Santiago de Cuba,Santiago de Cuba,Cuba,Caribbean,LATAM,TEC-CO-3709,Technology,Copiers,"Canon Wireless Fax, High-Speed","$1,002.95",4,0.002,$168.79,66.937,Medium
38311,39578,CA-2015-MC17605140-42134,5/10/15,5/11/15,First Class,MC-176051408,Matt Connell,Corporate,28540.0,Jacksonville,North Carolina,United States,Southern US,USCA,FUR-CH-4556,Furniture,Chairs,Harbour Creations Steel Folding Chair,$207.00,3,0.200,$25.88,63.050,Critical
24275,30535,IN-2013-AP1072092-41409,5/15/13,5/17/13,Second Class,AP-1072092,Anne Pryor,Home Office,NaN,New Plymouth,Taranaki,New Zealand,Oceania,Asia Pacific,OFF-EN-4904,Office Supplies,Envelopes,"Jiffy Business Envelopes, Security-Tint",$31.92,2,0.000,$6.06,8.700,Critical
24746,4964,US-2015-JC1538598-42102,4/8/15,4/12/15,Standard Class,JC-1538598,Jenna Caffey,Consumer,NaN,David,Chiriquí,Panama,Central America,LATAM,TEC-PH-3150,Technology,Phones,"Apple Speaker Phone, Cordless",$151.38,3,0.400,-$90.84,8.376,Medium
30958,24997,IN-2014-DP1339058-41880,8/29/14,9/2/14,Standard Class,DP-1339058,Dennis Pardue,Home Office,NaN,Kanpur,Uttar Pradesh,India,Southern Asia,Asia Pacific,OFF-SU-4995,Office Supplies,Supplies,"Kleencut Trimmer, Serrated",$114.39,3,0.000,$14.85,5.080,Medium
14811,916,MX-2012-DC1328582-41026,4/27/12,5/1/12,Standard Class,DC-1328582,Debra Catini,Consumer,NaN,Soledad Díez Gutiérrez,San Luis Potosí,Mexico,Central America,LATAM,TEC-PH-3140,Technology,Phones,"Apple Office Telephone, Full Size",$177.52,4,0.000,$7.04,20.052,High
1239,28645,IN-2012-BM1178558-41124,8/3/12,8/7/12,Standard Class,BM-1178558,Bryan Mills,Consumer,NaN,Tiruchchirappalli,Tamil Nadu,India,Southern Asia,Asia Pacific,FUR-BO-3639,Furniture,Bookcases,"Bush Library with Doors, Metal","$1,455.12",4,0.000,$116.40,179.620,Medium


In [10]:
returns.sample(10)

,Returned,Order ID,Region
1022,Yes,EG-2014-CS186038-41690,North Africa
444,Yes,IT-2014-TP2141545-41858,Western Europe
875,Yes,CA-2014-TB21280140-41786,Eastern US
532,Yes,SF-2015-RS9765117-42224,Southern Africa
232,Yes,CA-2012-BS11755140-41046,Eastern US
424,Yes,CA-2015-KH16330140-42329,Eastern US
351,Yes,BU-2012-EH394519-41265,Eastern Europe
956,Yes,CA-2015-HM14860140-42221,Western US
962,Yes,US-2014-LH1675055-41840,Central America
179,Yes,ES-2014-LC1705048-41718,Western Europe


In [22]:
type(orders.Profit)


pandas.core.series.Series

In [46]:
orders.Profit = orders.Profit.map(lambda x: x.replace('$',''))
orders.Profit = orders.Profit.map(lambda x: x.replace(',',''))
orders.Profit.astype(str).astype(float)


0          62.15
1        -288.77
2         919.97
3         -96.54
4         311.52
5         763.28
6         564.84
7         996.48
8          54.71
9           5.48
10          4.66
11        647.55
12       1151.40
13        622.02
14       1036.08
15        186.95
16         -2.52
17       3979.08
18         28.40
19        818.28
20        276.84
21          4.86
22        358.02
23       1898.40
24        445.52
25        526.50
26        720.36
27       1364.24
28        121.44
29        582.36
          ...   
51260      -1.11
51261      51.50
51262      16.01
51263       3.28
51264       8.86
51265     124.78
51266     -18.83
51267      16.59
51268       2.16
51269      12.00
51270       5.88
51271      22.05
51272       0.56
51273       0.04
51274       3.72
51275     -21.17
51276       7.38
51277     -49.57
51278      12.60
51279      -6.46
51280       5.00
51281       9.20
51282      19.95
51283       1.86
51284       8.00
51285       4.50
51286       3.34
51287       2.

In [47]:
orders.Sales = orders.Sales.map(lambda x: x.replace('$',''))
orders.Sales = orders.Sales.map(lambda x: x.replace(',',''))
orders.Sales.astype(str).astype(float)

0         221.98
1        3709.40
2        5175.17
3        2892.51
4        2832.96
5        2862.68
6        1822.08
7        5244.84
8         341.96
9          48.71
10         17.94
11       4626.15
12       2616.96
13       2221.80
14       3701.52
15       1869.59
16         12.62
17       7958.58
18       2565.59
19       3409.74
20       1977.72
21        242.94
22       2754.00
23       5273.70
24       1713.84
25       2106.50
26       1715.16
27       3069.74
28        247.84
29       1878.72
          ...   
51260       0.44
51261     823.96
51262     213.48
51263      36.48
51264      47.14
51265     259.96
51266      49.30
51267      61.44
51268       5.34
51269     200.40
51270      17.88
51271      71.12
51272      19.32
51273       5.92
51274      41.64
51275       9.61
51276      22.72
51277      34.13
51278      52.59
51279       5.36
51280      15.98
51281      84.00
51282      58.05
51283      26.94
51284      18.64
51285      65.10
51286      16.72
51287       8.